In [1]:
# importing required modules
from zipfile import ZipFile
import numpy as np
import pandas as pd
import os

In [ ]:
# # specifying the zip file name
# file_name = "sib_ativos.zip"
  
# # opening the zip file in READ mode
# with ZipFile(file_name, 'r') as zip:
#     # printing all the contents of the zip file
#     zip.printdir()
  
#     # extracting all the files
#     print('Extracting all the files now...')
#     zip.extractall()
#     print('Done!')

In [2]:
# specifying the zip file name
zipfile_name = "sib_ativos.zip"

# Colunas que serão utilizadas
cols = ['LG_BENEFICIARIO_ATIVO', 'DT_NASCIMENTO', 'TP_SEXO',    
       'DT_CONTRATACAO', 'ID_BENE_TIPO_DEPENDENTE', 'SG_UF',
       'LG_RESIDE_EXTERIOR', 'DT_CANCELAMENTO', 'CD_BENE_MOTIV_CANCELAMENTO']

datatypes = {"ID_BENE_TIPO_DEPENDENTE": str, "CD_BENE_MOTIV_CANCELAMENTO": str}

# opening the zip file in READ mode
with ZipFile(zipfile_name, 'r') as zip:
    # printing all the contents of the zip file
    files = zip.namelist()
    
    for file in files:
        with zip.open(file) as myfile:
            df = pd.read_csv(myfile, sep=';', encoding = 'cp1252', usecols=cols, dtype=datatypes)
            df.to_csv(file, sep=";")

: 

: 

### Tentando unir os Dataframes em um único arquivo .csv

In [8]:
files = ['sib_202211_AC.csv', 'sib_202211_AL.csv', 'sib_202211_AM.csv', 'sib_202211_AP.csv', 'sib_202211_BA.csv', 'sib_202211_CE.csv', 'sib_202211_DF.csv', 'sib_202211_ES.csv', 'sib_202211_GO.csv', 'sib_202211_MA.csv', 'sib_202211_MG.csv', 'sib_202211_MS.csv', 'sib_202211_MT.csv', 'sib_202211_PA.csv', 'sib_202211_PB.csv', 'sib_202211_PE.csv', 'sib_202211_PI.csv', 'sib_202211_PR.csv', 'sib_202211_RJ.csv', 'sib_202211_RN.csv', 'sib_202211_RO.csv', 'sib_202211_RR.csv', 'sib_202211_RS.csv', 'sib_202211_SC.csv', 'sib_202211_SE.csv', 'sib_202211_SP.csv', 'sib_202211_TO.csv']
cols = ['LG_BENEFICIARIO_ATIVO', 'DT_NASCIMENTO', 'TP_SEXO',    
       'DT_CONTRATACAO', 'ID_BENE_TIPO_DEPENDENTE', 'CD_MUNICIPIO', 'SG_UF',
       'LG_RESIDE_EXTERIOR', 'DT_CANCELAMENTO', 'CD_BENE_MOTIV_CANCELAMENTO']

datatypes = {"ID_BENE_TIPO_DEPENDENTE": str, "CD_BENE_MOTIV_CANCELAMENTO": str}

# DF base
df = pd.DataFrame(columns= cols)

for file in files:
    df_ = pd.read_csv(file, sep=';', encoding = 'cp1252', usecols=cols, dtype=datatypes)
    df = pd.concat([df, df_], ignore_index=True)
    df_.to_csv("sib_202211.csv", sep=";")

: 

: 

In [ ]:
# Consulta arquivos na pasta
os.listdir()

In [2]:
cols = ['LG_BENEFICIARIO_ATIVO', 'DT_NASCIMENTO', 'TP_SEXO',    
       'DT_CONTRATACAO', 'ID_BENE_TIPO_DEPENDENTE', 'CD_MUNICIPIO', 'SG_UF',
       'LG_RESIDE_EXTERIOR', 'DT_CANCELAMENTO', 'CD_BENE_MOTIV_CANCELAMENTO']

datatypes = {"ID_BENE_TIPO_DEPENDENTE": str, "CD_BENE_MOTIV_CANCELAMENTO": str}

df1 = pd.read_csv('sib_202211_AC.csv', sep=';', encoding = 'cp1252', usecols=cols, dtype=datatypes, na_values='-')
df2 = pd.read_csv('sib_202211_TO.csv', sep=';', encoding = 'cp1252', usecols=cols, dtype=datatypes, na_values='-')

In [6]:
df = pd.concat([df1,df2], ignore_index=True)

df

,Unnamed: 0,LG_BENEFICIARIO_ATIVO,DT_NASCIMENTO,TP_SEXO,DT_CONTRATACAO,ID_BENE_TIPO_DEPENDENTE,CD_MUNICIPIO,SG_UF,LG_RESIDE_EXTERIOR,DT_CANCELAMENTO,CD_BENE_MOTIV_CANCELAMENTO
0,0.0,1,1953.0,F,2000-11,1.0,521310,GO,0.0,NaN,NaN
1,1.0,1,1984.0,M,2002-12,1.0,521310,GO,0.0,NaN,NaN
2,2.0,1,1958.0,M,2000-08,1.0,521310,GO,0.0,2007-07,41.0
3,3.0,1,1998.0,F,2005-12,1.0,521310,GO,0.0,2021-01,43.0
4,4.0,1,1968.0,F,2003-09,1.0,521310,GO,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2423479,NaN,1,1965.0,M,2022-07,3,172100,TO,0.0,NaN,NaN
2423480,NaN,1,1996.0,F,2019-11,4,172100,TO,0.0,NaN,NaN
2423481,NaN,1,1994.0,F,2022-08,3,172100,TO,0.0,NaN,NaN
2423482,NaN,1,1958.0,F,2022-08,3,172100,TO,0.0,NaN,NaN
